In [1]:
##IMPORTING LIBRARIES
import pandas as pd
import statistics as stats
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Suppress the warnings
import warnings
warnings.filterwarnings('ignore')

In [14]:
#Creating dataframes from clean files
df_pop = pd.read_csv("PEA_c.csv")
df_f = pd.read_csv("F2002_c.csv")
df_ica = pd.read_csv("ICA_c.csv")

df_pop.head()

,Year,Region,Estimated Population nr
0,2007,West,846400.0
1,2007,Dublin,2462500.0
2,2007,Mid-East,992100.0
3,2007,Mid-West,733000.0
4,2007,Midlands,524600.0


In [15]:
df_pop.shape

(153, 3)

In [16]:
df_pop.Region.unique()

array(['West', 'Dublin', 'Mid-East', 'Mid-West', 'Midlands', 'South-East',
       'South-West', 'Ireland', 'Border'], dtype=object)

In [17]:
df_f.head()

,Year,Region,Average Nr of Persons Household
0,1971,Border,3.727
1,1971,South-West,3.910
2,1971,Ireland,3.940
3,1971,South-East,4.047
4,1971,Mid-East,4.127


In [18]:
df_f.shape

(189, 3)

In [19]:
df_f.Region.unique()

array(['Border', 'South-West', 'Ireland', 'South-East', 'Mid-East',
       'Mid-West', 'Midlands', 'Dublin', 'West'], dtype=object)

In [20]:
df_ica.head()

,Year,Region,Households with Internet Access
0,2007,Border,43.0
1,2007,Midlands,54.0
2,2007,South-East,49.0
3,2007,Mid-West,58.0
4,2007,South-West,56.0


In [21]:
df_ica.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 3 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Year                             144 non-null    int64  
 1   Region                           144 non-null    object 
 2   Households with Internet Access  144 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 3.5+ KB


In [22]:
df_ica.shape

(144, 3)

In [23]:
df_ica.Region.unique()

array(['Border', 'Midlands', 'South-East', 'Mid-West', 'South-West',
       'Mid-East', 'Ireland', 'Dublin', 'West'], dtype=object)

In [30]:
df_agg1 = df_pop.merge(df_f, on=['Year', 'Region'], how='left')
df_agg1.head()

,Year,Region,Estimated Population nr,Average Nr of Persons Household
0,2007,West,846400.0,2.767
1,2007,Dublin,2462500.0,2.714
2,2007,Mid-East,992100.0,2.959
3,2007,Mid-West,733000.0,2.770
4,2007,Midlands,524600.0,2.858


In [31]:
df_agg1 = df_agg1.merge(df_ica, on=['Year', 'Region'], how='left')
df_agg1.head()

,Year,Region,Estimated Population nr,Average Nr of Persons Household,Households with Internet Access
0,2007,West,846400.0,2.767,52.0
1,2007,Dublin,2462500.0,2.714,66.0
2,2007,Mid-East,992100.0,2.959,66.0
3,2007,Mid-West,733000.0,2.770,58.0
4,2007,Midlands,524600.0,2.858,54.0


In [32]:
df_agg1['Nr of Households'] = df_agg1['Estimated Population nr'] / df_agg1['Average Nr of Persons Household']

In [34]:
df_agg1['Nr of Households with Internet Access'] = df_agg1['Nr of Households']*df_agg1['Households with Internet Access']/100

In [40]:
df_agg1['Nr of Persons with Internet Access'] = df_agg1['Nr of Households with Internet Access'] * df_agg1['Average Nr of Persons Household']

In [41]:
df_agg1.head()

,Year,Region,Estimated Population nr,Average Nr of Persons Household,Households with Internet Access,Nr of Households,Nr of Households with Internet Access,Nr of Persons with Internet Access
0,2007,West,846400.0,2.767,52.0,305890.856523,159063.245392,440128.0
1,2007,Dublin,2462500.0,2.714,66.0,907332.350774,598839.351511,1625250.0
2,2007,Mid-East,992100.0,2.959,66.0,335282.189929,221286.245353,654786.0
3,2007,Mid-West,733000.0,2.770,58.0,264620.938628,153480.144404,425140.0
4,2007,Midlands,524600.0,2.858,54.0,183554.933520,99119.664101,283284.0


In [43]:
df_agg1.to_csv("FINAL.csv", index = False)